In [1]:
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# setting size of screen & title
screen_title = 'Angie\'s Adventure'
screen_width = 800
screen_height = 800

In [3]:
# start game
pygame.init()

(6, 0)

In [4]:
# colors using Red Green Blue codes RGB 
white_color = (255,255,255)
green_color = (0,255,0)
blue_color = (0,0,255)
red_color = (255,0,0)
black_color = (0,0,0)

In [5]:
# Creating game clock
clock = pygame.time.Clock()

In [6]:
# End game display font
pygame.font.init()
font = pygame.font.SysFont('comicsans', 100)

In [7]:
# Creating the game class loop
class Game:

    # Typical rate of 60, equivalents to Frames Per Second
    tick_rate = 60
    is_game_over = False

    def __init__(self, image_path, title, width, height):
        self.title = title
        self.width = width
        self.height = height
        
        # Create the window of specified size in white to display the game
        self.game_screen = pygame.display.set_mode((width, height))

        # Set the game window color to white
        self.game_screen.fill(white_color)
        pygame.display.set_caption(title)

        background_image = pygame.image.load(image_path)
        self.image = pygame.transform.scale(background_image, (width, height))

    def run_game_loop(self,):
        is_game_over = False
        direction_x = 0
        direction_y = 0
        
        player_character = PlayerCharacter('smiley_angie.jpeg', 375, 740, 50, 50)
        non_player_character = NonPlayerCharacter('frank_closeup.jpeg', 20, 450, 100, 100)
        treasure = GameObject('treat.jpeg', 375, 20, 40, 40)
        # Main game loop, used to update all gameplay (movement, checks, graphics)
        # Runs until is_game_over = True
        while not is_game_over:

            # A loop to get all of the events occuring at a given time
            # Events are most often mouse movement, mouse and button clicks, or exit events
            for event in pygame.event.get():
                # If we have a quit type event (exit out) the exit out of the game loop
                if event.type == pygame.QUIT:
                    is_game_over = True
                # Detect when a key is pressed down
                elif event.type == pygame.KEYDOWN:
                    # Move up if up key is pressed
                    if event.key == pygame.K_UP:
                        direction_y = 1
                    # Move down if down key is pressed
                    elif event.key == pygame.K_DOWN:
                        direction_y = -1
                    # Move right if right key is pressed
                    elif event.key == pygame.K_RIGHT:
                        direction_x = -1
                    # Move left if left key is pressed
                    elif event.key == pygame.K_LEFT:
                        direction_x = 1
                # Detect when the key is released
                elif event.type == pygame.KEYUP:
                    # Stop movement when the key is released
                    if event.key == pygame.K_UP or event.key == pygame.K_DOWN or event.key == pygame.K_RIGHT or event.key == pygame.K_LEFT:
                        direction_y = 0
                        direction_x = 0
                print(event)

            # Redraw the screen to be a blank white window
            self.game_screen.fill(white_color)
            self.game_screen.blit(self.image, (0,0))

            # Draw the treasure
            treasure.draw(self.game_screen)
            
            # Update the player position
            player_character.move_y(direction_y, self.height)
            player_character.move_x(direction_x, self.width)
            # Draw the player at new postion
            player_character.draw(self.game_screen)

            # Draw NPC at start positition and move character
            non_player_character.move(self.width)
            non_player_character.draw(self.game_screen)

            # End game is collision happens with NPC or treasure
            if player_character.detect_collision(non_player_character):
                is_game_over = True
                did_win = False
                text = font.render('You Lose', True, black_color)
                self.game_screen.blit(text,(300,350))
                pygame.display.update()
                clock.tick(1)
                break
            elif player_character.detect_collision(treasure):
                is_game_over = True
                did_win = True
                text = font.render('You Win', True, black_color)
                self.game_screen.blit(text,(300,350))
                pygame.display.update()
                clock.tick(1)
                break
            # Update all game graphics
            pygame.display.update()
            # Tick the clock to update everything with in the game
            clock.tick(self.tick_rate)

        if did_win:
            self.run_game_loop()
        else:
            return

In [8]:
# Class to represent any object in the game
class GameObject:

    def __init__(self, image_path, x, y, width, height):

        # Load object image
        object_image = pygame.image.load(image_path)
        # Scale object image
        self.image = pygame.transform.scale(object_image, (width, height))
        
        self.x_pos = x
        self.y_pos = y

        self.width = width
        self.height = height

    # Draw object by blitting it onto the background (game screen)
    def draw(self, background):
        background.blit(self.image, (self.x_pos, self.y_pos))

In [9]:
# Class to represent the character controlled by the player
class PlayerCharacter(GameObject):

    # How many tiles the character moves per secon
    Speed = 10
    
    def __init__(self, image_path, x, y, width, height):
        super().__init__(image_path, x, y, width, height)

    # Move function will move character up if direction is > 0 and down if < 0
    def move_y(self, direction_y, max_height):
        if direction_y > 0:
            self.y_pos -= self.Speed
        elif direction_y < 0:
            self.y_pos += self.Speed
        # Make sure the character never goes past the bottom of the screen
        if self.y_pos >= max_height - 60:
            self.y_pos = max_height - 60
        # Make sure the character never goes past the top of the screen
        elif self.y_pos <= 10:
            self.y_pos = 10
    def move_x(self, direction_x, max_width):
        if direction_x > 0:
            self.x_pos -= self.Speed
        elif direction_x < 0:
            self.x_pos += self.Speed
        # Make sure the character never goes past the right side of the screen
        if self.x_pos >= max_width - 60:
            self.x_pos = max_width - 60
        # Make sure the character never goes past the left side of the screen
        elif self.x_pos <= 10:
            self.x_pos = 10
    def detect_collision(self, other_body):
        # Checking for Y axis overlap
        if self.y_pos > other_body.y_pos + other_body.height:
            return False
        elif self.y_pos + self.height < other_body.y_pos:
            return False
        # Checking for X axis overlap
        if self.x_pos > other_body.x_pos + other_body.width:
            return False
        elif self.x_pos + self.width < other_body.x_pos:
            return False
        # Only returns True when there is an overlap on both axis
        return True

In [10]:
# NPC creation (Enemy Characters)
# Class to represent the characters NOT controlled by the player
class NonPlayerCharacter(GameObject):

    # How many tiles the character moves per second
    Speed = 10
    
    def __init__(self, image_path, x, y, width, height):
        super().__init__(image_path, x, y, width, height)

    # Move function will move character up if direction is > 0 and down if < 0
    def move(self, max_width):
        if self.x_pos <= 20:
            self.Speed = abs(self.Speed)
        elif self.x_pos >= max_width - 40:
            self.Speed = -abs(self.Speed)
        self.x_pos += self.Speed

In [11]:
# start game
pygame.init()

new_game = Game('homebrew_background.png', screen_title,  screen_width, screen_height)
new_game.run_game_loop()


<Event(17-VideoExpose {})>
<Event(16-VideoResize {'size': (800, 800), 'w': 800, 'h': 800})>
<Event(1-ActiveEvent {'gain': 0, 'state': 1})>
<Event(4-MouseMotion {'pos': (0, 0), 'rel': (1, 1), 'buttons': (0, 0, 0), 'window': None})>
<Event(1-ActiveEvent {'gain': 1, 'state': 1})>
<Event(4-MouseMotion {'pos': (799, 498), 'rel': (799, 498), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (782, 471), 'rel': (-17, -27), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (765, 440), 'rel': (-17, -31), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (742, 400), 'rel': (-23, -40), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (719, 362), 'rel': (-23, -38), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (698, 332), 'rel': (-21, -30), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {'pos': (682, 308), 'rel': (-16, -24), 'buttons': (0, 0, 0), 'window': None})>
<Event(4-MouseMotion {

In [12]:
# Quit the game and program
pygame.quit()
quit()